# Django Forms та обробка POST-запитів

## Мета лекції

Після цієї лекції ви зможете:

* зрозуміти різницю між GET та POST;
* створювати HTML-форми в Django;
* обробляти POST-запити у view;
* використовувати Django Forms;
* виконувати валідацію даних;
* безпечно працювати з CSRF;
* застосовувати best practices.

---

## 1. GET vs POST — базова теорія

### GET

* використовується для отримання даних
* параметри передаються в URL
* **небезпечно** для паролів

```text
/login/?username=admin
```

### POST

* використовується для відправки даних
* дані передаються в тілі запиту
* безпечніше

📌 **Форми з даними → POST**

---

## 2. Проста HTML-форма

```html
<form method="post">
    <input type="text" name="username">
    <button type="submit">Send</button>
</form>
```

❗ Без Django це працює, але **без валідації та захисту**.

---

## 3. CSRF-захист у Django

Django **автоматично захищає** POST-запити.

### У шаблоні обовʼязково:

```html
<form method="post">
    {% csrf_token %}
    <input type="text" name="username">
    <button type="submit">Send</button>
</form>
```

❌ Без `{% csrf_token %}` → **403 Forbidden**

---

## 4. Обробка POST у view (без Forms)

```python
from django.http import HttpResponse

def simple_form(request):
    if request.method == 'POST':
        username = request.POST.get('username')
        return HttpResponse(f"Hello, {username}")

    return HttpResponse("GET request")
```

⚠️ Мінус: **немає валідації**

---

## 5. Django Forms — навіщо вони?

Django Forms:

* автоматично валідуюють дані
* захищають від помилок
* спрощують HTML
* зменшують boilerplate

📌 **Forms = must-have у реальних проєктах**

---

## 6. Створення Django Form

### forms.py

```python
from django import forms

class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    message = forms.CharField(widget=forms.Textarea)
```

---

## 7. View з Django Form

```python
from django.shortcuts import render
from .forms import ContactForm

def contact_view(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            name = form.cleaned_data['name']
            return render(request, 'success.html', {'name': name})
    else:
        form = ContactForm()

    return render(request, 'contact.html', {'form': form})
```

📌 `is_valid()` запускає валідацію

---

## 8. Вивід форми у шаблоні

```html
<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Send</button>
</form>
```

### Інші варіанти

* `form.as_table`
* `form.as_ul`

---

## 9. Валідація та помилки

```html
{% if form.errors %}
    {{ form.errors }}
{% endif %}
```

### Кастомна валідація

```python
class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)

    def clean_name(self):
        name = self.cleaned_data['name']
        if 'admin' in name.lower():
            raise forms.ValidationError("Invalid name")
        return name
```

---

## 10. cleaned_data

```python
form.cleaned_data
```

➡ містить **тільки валідні дані**

---

## 11. Redirect після POST (PRG pattern)

❌ Погано:

```python
return render(request, 'success.html')
```

✅ Добре:

```python
from django.shortcuts import redirect

return redirect('success')
```

📌 **POST → Redirect → GET**

---

## 12. ModelForm — короткий вступ

```python
from django.forms import ModelForm
from .models import Post

class PostForm(ModelForm):
    class Meta:
        model = Post
        fields = ['title', 'content']
```

📌 ModelForm = Form + Model

---

## 13. Типові помилки

❌ Забули `{% csrf_token %}`

❌ Не перевірили `is_valid()`

❌ Використали GET для форм

❌ Логіка у шаблоні

---

## 14. Коротке резюме

* POST — для даних
* Django Forms = стандарт
* `is_valid()` — обовʼязково
* `cleaned_data` — безпечні дані
* PRG — best practice

---

## Практичне завдання

1. Створіть форму з полями: name, email
2. Додайте валідацію email
3. Обробіть POST
4. Зробіть redirect після успіху

---

📌 Наступна лекція: **ModelForm + CRUD у Django**
